In [0]:
import logging 
from log_utils import setup_logging
from datetime import datetime

In [0]:
logger = logging.getLogger(__name__)
setup_logging()

In [0]:
source_catalog = 'samples'
source_schema = 'bakehouse'

target_catalog = 'analytics'
target_schema = 'bakehouse_raw'


tables_to_be_ingested = [
    'sales_transactions'
    ,'sales_suppliers'
    ,'sales_franchises'
    ,'sales_customers'
    ,'media_gold_reviews_chunked'
    ,'media_customer_reviews'
]

In [0]:
job_start_date = datetime.now()

In [0]:
from pyspark.sql.functions import to_timestamp, lit

#ingest from one catalog to another catalog
for table in tables_to_be_ingested:
    try:
        df = (
            spark.table(f'{source_catalog}.{source_schema}.{table}')
            .withColumn(
                '_ingested_at',
                to_timestamp(lit(job_start_date.isoformat()))
            )
        )
        logger.info(f'Ingesting table {table}')
        df.write.saveAsTable(
            f'{target_catalog}.{target_schema}.{table}',
            mode='append'
        )
        logger.info(f'Ingested table {table}')
    except:
        logger.exception(f'Failed to ingest table')